# Intro to Keras using MNIST

In [1]:
import numpy as np
np.random.seed(123)

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

from keras.datasets import mnist

# Load pre-shuffled MNIST data into train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

Using TensorFlow backend.


### Reshape data - 1 layer (grayscale), then 28x28 pixels

Using TensorFlow, images are represented as NumPy arrays with the shape (height, width, depth), where the depth is the number of channels in the image. However, if you are using Theano, images are instead assumed to be represented as (depth, height, width).

In [2]:
img_rows = 28
img_cols = 28

if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)

X_train.shape

(60000, 28, 28, 1)

### Convert data to float32 and normalize from 0-1

In [3]:
# convert data to float32 and normalize from 0-1
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

### Labels Data

In [4]:
print(y_train.shape)
print(y_train[:10])

(60000,)
[5 0 4 1 9 2 1 3 1 4]


### Convert 1-dimensional class arrays to 10-dimensional class matrices

In [5]:
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)
print(Y_train.shape)

(60000, 10)


### Model

#### Input layer
The input shape parameter should be the shape of 1 sample. In this case, it's the same (1, 28, 28) that corresponds to  the (depth, width, height) of each digit image.

But what do the first 3 parameters represent? They correspond to the number of convolution filters to use, the number of rows in each convolution kernel, and the number of columns in each convolution kernel, respectively.

Our CONV  layer will learn 20 convolution filters, where each filter is of size 5 x 5. The input dimensions of this value are the same width, height, and depth as our input images — in this case, the MNIST dataset, so we’ll have 28 x 28 inputs with a single channel for depth (grayscale).

We’ll then apply the ReLU activation function followed by 2 x 2 max-pooling in both the x and y direction with a stride of 2 (imagine a 2 x 2 sliding window that “slides” across the activation volume, taking the max operation of each region, while taking a step of 2 pixels in both the horizontal and vertical direction).

It’s common to see the number of CONV  filters learned increase in deeper layers of the network.

In [6]:
model = Sequential()
# input layer
#model.add(Convolution2D(32, 3, 3, activation='relu', input_shape=(1,28,28))) # if theano backend
model.add(Convolution2D(32, 3, 3, activation='relu', input_shape=(28, 28, 1)))

In [7]:
model.output_shape

(None, 26, 26, 32)

### More layers
We won't go into the theory too much, but it's important to highlight the Dropout layer we just added. This is a method for regularizing our model in order to prevent overfitting.

MaxPooling2D is a way to reduce the number of parameters in our model by sliding a 2x2 pooling filter across the previous layer and taking the max of the 4 values in the 2x2 filter.

we take the output of the preceding  MaxPooling2D  layer and flatten it into a single vector, allowing us to apply dense/fully-connected layers. If you have any prior experience with neural networks, then you’ll know that a dense/fully-connected layer is a “standard” type of layer in a network, where every node in the preceding layer connects to every node in the next layer (hence the term, “fully-connected”).

In [8]:
model.add(Convolution2D(32, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

### Even more layers
So far, for model parameters, we've added two Convolution layers. To complete our model architecture, let's add a fully connected layer and then the output layer.

For Dense layers, the first parameter is the output size of the layer. Keras automatically handles the connections between layers.

Note that the final layer has an output size of 10, corresponding to the 10 classes of digits.

Also note that the weights from the Convolution layers must be flattened (made 1-dimensional) before passing them to the fully connected Dense layer.

In [9]:
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

### Compile model

In [10]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Fit model on training data

In [11]:
model.fit(X_train, Y_train, batch_size=32, nb_epoch=10, verbose=1)

Epoch 1/10
60000/60000 [==============================] - 48s - loss: 0.2032 - acc: 0.9380    
Epoch 2/10
60000/60000 [==============================] - 42s - loss: 0.0840 - acc: 0.9748    
Epoch 3/10
60000/60000 [==============================] - 42s - loss: 0.0653 - acc: 0.9804    
Epoch 4/10
60000/60000 [==============================] - 43s - loss: 0.0549 - acc: 0.9829    
Epoch 5/10
60000/60000 [==============================] - 42s - loss: 0.0493 - acc: 0.9848    
Epoch 6/10
60000/60000 [==============================] - 45s - loss: 0.0428 - acc: 0.9867    
Epoch 7/10
60000/60000 [==============================] - 46s - loss: 0.0363 - acc: 0.9881    
Epoch 8/10
60000/60000 [==============================] - 47s - loss: 0.0336 - acc: 0.9897    
Epoch 9/10
60000/60000 [==============================] - 47s - loss: 0.0312 - acc: 0.9900    
Epoch 10/10
60000/60000 [==============================] - 49s - loss: 0.0282 - acc: 0.9911    


### Evaluate model

In [12]:
score = model.evaluate(X_test, Y_test, verbose=0)
score

[0.028570816709818883, 0.99209999999999998]